In [90]:
# from google.colab import drive


# drive.mount('/content/gdrive')

In [91]:
# cd gdrive/My Drive/Colab Notebooks/project_paylocity/ashrae-energy-prediction

In [150]:
import os
os.listdir()

['.local',
 'train_preprocessed_may.csv',
 '.conda',
 '.ssh',
 'may__.ipynb',
 '.ipynb_checkpoints',
 'lr_find_temp_model_b2cf9b09-a57f-4035-ac29-ea88053e48a1.ckpt',
 'README',
 '.bashrc',
 '.keras',
 '.dl_binaries',
 '.cache',
 'lightning_logs',
 '.ipython',
 'src',
 '.bash_history',
 'tools',
 '.safety',
 '.profile',
 'LICENSE',
 'Nvidia_Cloud_EULA.pdf',
 '.sudo_as_admin_successful',
 '.jupyter',
 'examples',
 'nvidia-acknowledgements',
 '.condarc',
 'tutorials',
 '.config',
 '.bash_logout',
 'anaconda3',
 '.gnupg']

In [151]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast
from collections import Counter
import datetime
import seaborn as sns

# import lightgbm as lgb
# import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import datetime
import tqdm

from sklearn.metrics import mean_squared_log_error
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from pandas.plotting import autocorrelation_plot

In [152]:
train_preprocessed_may = pd.read_csv('train_preprocessed_may.csv')

In [153]:
train_2 = train_preprocessed_may[train_preprocessed_may['site_id'].isin([12,11])]

In [154]:
train_2["timestamp"] = pd.to_datetime(train_2["timestamp"])
train_2["hour"] = train_2["timestamp"].dt.hour
train_2["day"] = train_2["timestamp"].dt.day
train_2["weekend"] = train_2["timestamp"].dt.weekday
train_2["month"] = train_2["timestamp"].dt.month

train_2 = train_2.reset_index()
train_2 = train_2.drop(columns = ['index'], axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [155]:
# train_final = train_2.head(int(len(train_2)*(90/100)))
# test_final = train_2[len(train_final):]

In [156]:
def add_lags(df, window=3):
    group_df = df.groupby(['building_id','site_id'])
    cols = ['air_temperature', 'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction', 'wind_speed']
    rolled = group_df[cols].rolling(window=window, min_periods=0)
    lag_mean = rolled.mean().reset_index().astype(np.float16)
    lag_max = rolled.max().reset_index().astype(np.float16)
    lag_min = rolled.min().reset_index().astype(np.float16)
    lag_std = rolled.std().reset_index().astype(np.float16)
    for col in cols:
        df[f'{col}_mean_lag{window}'] = lag_mean[col]
        df[f'{col}_max_lag{window}'] = lag_max[col]
        df[f'{col}_min_lag{window}'] = lag_min[col]
        df[f'{col}_std_lag{window}'] = lag_std[col]

In [157]:
add_lags(train_2, window = 3)

In [158]:
train_2.head(3)

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,day,weekend,month,air_temperature_mean_lag3,air_temperature_max_lag3,air_temperature_min_lag3,air_temperature_std_lag3,cloud_coverage_mean_lag3,cloud_coverage_max_lag3,cloud_coverage_min_lag3,cloud_coverage_std_lag3,dew_temperature_mean_lag3,dew_temperature_max_lag3,dew_temperature_min_lag3,dew_temperature_std_lag3,precip_depth_1_hr_mean_lag3,precip_depth_1_hr_max_lag3,precip_depth_1_hr_min_lag3,precip_depth_1_hr_std_lag3,sea_level_pressure_mean_lag3,sea_level_pressure_max_lag3,sea_level_pressure_min_lag3,sea_level_pressure_std_lag3,wind_direction_mean_lag3,wind_direction_max_lag3,wind_direction_min_lag3,wind_direction_std_lag3,wind_speed_mean_lag3,wind_speed_max_lag3,wind_speed_min_lag3,wind_speed_std_lag3
0,1029,0,2016-01-01,246.000,11,Education,152559,1970.0,3.0,-1.8,9.0,-3.2,0.0,1016.0,280.0,1.5,0,1,4,1,-14.703125,-14.703125,-14.703125,NaN,9.0,9.0,9.0,NaN,-16.40625,-16.40625,-16.40625,NaN,2.0,2.0,2.0,NaN,1008.5,1008.5,1008.5,NaN,60.0,60.0,60.0,NaN,2.099609,2.099609,2.099609,NaN
1,1029,3,2016-01-01,140.929,11,Education,152559,1970.0,3.0,-1.8,9.0,-3.2,0.0,1016.0,280.0,1.5,0,1,4,1,-14.703125,-14.703125,-14.703125,0.0,9.0,9.0,9.0,0.0,-16.40625,-16.40625,-16.40625,0.0,2.0,2.0,2.0,0.0,1008.5,1008.5,1008.5,0.0,60.0,60.0,60.0,0.0,2.099609,2.099609,2.099609,0.0
2,1030,0,2016-01-01,50.000,11,Education,68030,1970.0,3.0,-1.8,9.0,-3.2,0.0,1016.0,280.0,1.5,0,1,4,1,-14.703125,-14.703125,-14.703125,0.0,9.0,9.0,9.0,0.0,-16.40625,-16.40625,-16.40625,0.0,2.0,2.0,2.0,0.0,1008.5,1008.5,1008.5,0.0,60.0,60.0,60.0,0.0,2.099609,2.099609,2.099609,0.0


In [159]:
# def features_engineering3(df):
    
#     df.sort_values("timestamp")
#     df.reset_index(drop=True)
    
#     df["timestamp"] = pd.to_datetime(df["timestamp"],format="%Y-%m-%d %H:%M:%S")
#     df['square_feet'] =  np.log1p(df['square_feet'])

#     # Remove unimp Columns
#     drop = ["timestamp"]
#     df = df.drop(drop, axis=1)

#     le = LabelEncoder()
#     df["primary_use"] = le.fit_transform(df["primary_use"])
    
#     return df

In [160]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [161]:
!pip install pytorch-forecasting

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [162]:
import copy
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

In [163]:
from pytorch_forecasting.data.examples import get_stallion_data




In [164]:
train_2.head(1)

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,day,weekend,month,air_temperature_mean_lag3,air_temperature_max_lag3,air_temperature_min_lag3,air_temperature_std_lag3,cloud_coverage_mean_lag3,cloud_coverage_max_lag3,cloud_coverage_min_lag3,cloud_coverage_std_lag3,dew_temperature_mean_lag3,dew_temperature_max_lag3,dew_temperature_min_lag3,dew_temperature_std_lag3,precip_depth_1_hr_mean_lag3,precip_depth_1_hr_max_lag3,precip_depth_1_hr_min_lag3,precip_depth_1_hr_std_lag3,sea_level_pressure_mean_lag3,sea_level_pressure_max_lag3,sea_level_pressure_min_lag3,sea_level_pressure_std_lag3,wind_direction_mean_lag3,wind_direction_max_lag3,wind_direction_min_lag3,wind_direction_std_lag3,wind_speed_mean_lag3,wind_speed_max_lag3,wind_speed_min_lag3,wind_speed_std_lag3
0,1029,0,2016-01-01,246.0,11,Education,152559,1970.0,3.0,-1.8,9.0,-3.2,0.0,1016.0,280.0,1.5,0,1,4,1,-14.703125,-14.703125,-14.703125,NaN,9.0,9.0,9.0,NaN,-16.40625,-16.40625,-16.40625,NaN,2.0,2.0,2.0,NaN,1008.5,1008.5,1008.5,NaN,60.0,60.0,60.0,NaN,2.099609,2.099609,2.099609,NaN


In [165]:
train_2.shape

(435368, 48)

In [166]:
train_2.shape

(435368, 48)

In [168]:
train_2["building_id"] = train_2["building_id"].astype(str)
train_2["meter"] = train_2["meter"].astype(str) 
train_2["site_id"] = train_2["site_id"].astype(str) 

In [172]:
train_2.dtypes

building_id                             object
meter                                   object
timestamp                       datetime64[ns]
meter_reading                          float64
site_id                                 object
primary_use                             object
square_feet                              int64
year_built                             float64
floor_count                            float64
air_temperature                        float64
cloud_coverage                         float64
dew_temperature                        float64
precip_depth_1_hr                      float64
sea_level_pressure                     float64
wind_direction                         float64
wind_speed                             float64
hour                                     int64
day                                      int64
weekend                                  int64
month                                    int64
air_temperature_mean_lag3              float16
air_temperatu

In [175]:
train_2 = train_2[train_2.site_id == "11"]

In [185]:
data = train_2

In [186]:
data.head(2)

,building_id,meter,date,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,day,weekend,month,air_temperature_mean_lag3,air_temperature_max_lag3,air_temperature_min_lag3,air_temperature_std_lag3,cloud_coverage_mean_lag3,cloud_coverage_max_lag3,cloud_coverage_min_lag3,cloud_coverage_std_lag3,dew_temperature_mean_lag3,dew_temperature_max_lag3,dew_temperature_min_lag3,dew_temperature_std_lag3,precip_depth_1_hr_mean_lag3,precip_depth_1_hr_max_lag3,precip_depth_1_hr_min_lag3,precip_depth_1_hr_std_lag3,sea_level_pressure_mean_lag3,sea_level_pressure_max_lag3,sea_level_pressure_min_lag3,sea_level_pressure_std_lag3,wind_direction_mean_lag3,wind_direction_max_lag3,wind_direction_min_lag3,wind_direction_std_lag3,wind_speed_mean_lag3,wind_speed_max_lag3,wind_speed_min_lag3,wind_speed_std_lag3,time_idx,log_meter_reading
0,1029,0,2016-01-01,246.000,11,Education,152559,1970.0,3.0,-1.8,9.0,-3.2,0.0,1016.0,280.0,1.5,0,1,4,1,-14.703125,-14.703125,-14.703125,NaN,9.0,9.0,9.0,NaN,-16.40625,-16.40625,-16.40625,NaN,2.0,2.0,2.0,NaN,1008.5,1008.5,1008.5,NaN,60.0,60.0,60.0,NaN,2.099609,2.099609,2.099609,NaN,0,5.509388
1,1029,3,2016-01-01,140.929,11,Education,152559,1970.0,3.0,-1.8,9.0,-3.2,0.0,1016.0,280.0,1.5,0,1,4,1,-14.703125,-14.703125,-14.703125,0.0,9.0,9.0,9.0,0.0,-16.40625,-16.40625,-16.40625,0.0,2.0,2.0,2.0,0.0,1008.5,1008.5,1008.5,0.0,60.0,60.0,60.0,0.0,2.099609,2.099609,2.099609,0.0,0,4.955327


In [187]:
data.rename(columns = {'timestamp':'date'}, inplace = True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [188]:

data["time_idx"] = data["date"].dt.year * 12 + data["date"].dt.month
data["time_idx"] -= data["time_idx"].min()
data["month"] = data.date.dt.month.astype(str).astype("category") 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [189]:
data.shape

(119459, 50)

In [190]:
data.head(2)

,building_id,meter,date,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,day,weekend,month,air_temperature_mean_lag3,air_temperature_max_lag3,air_temperature_min_lag3,air_temperature_std_lag3,cloud_coverage_mean_lag3,cloud_coverage_max_lag3,cloud_coverage_min_lag3,cloud_coverage_std_lag3,dew_temperature_mean_lag3,dew_temperature_max_lag3,dew_temperature_min_lag3,dew_temperature_std_lag3,precip_depth_1_hr_mean_lag3,precip_depth_1_hr_max_lag3,precip_depth_1_hr_min_lag3,precip_depth_1_hr_std_lag3,sea_level_pressure_mean_lag3,sea_level_pressure_max_lag3,sea_level_pressure_min_lag3,sea_level_pressure_std_lag3,wind_direction_mean_lag3,wind_direction_max_lag3,wind_direction_min_lag3,wind_direction_std_lag3,wind_speed_mean_lag3,wind_speed_max_lag3,wind_speed_min_lag3,wind_speed_std_lag3,time_idx,log_meter_reading
0,1029,0,2016-01-01,246.000,11,Education,152559,1970.0,3.0,-1.8,9.0,-3.2,0.0,1016.0,280.0,1.5,0,1,4,1,-14.703125,-14.703125,-14.703125,NaN,9.0,9.0,9.0,NaN,-16.40625,-16.40625,-16.40625,NaN,2.0,2.0,2.0,NaN,1008.5,1008.5,1008.5,NaN,60.0,60.0,60.0,NaN,2.099609,2.099609,2.099609,NaN,0,5.509388
1,1029,3,2016-01-01,140.929,11,Education,152559,1970.0,3.0,-1.8,9.0,-3.2,0.0,1016.0,280.0,1.5,0,1,4,1,-14.703125,-14.703125,-14.703125,0.0,9.0,9.0,9.0,0.0,-16.40625,-16.40625,-16.40625,0.0,2.0,2.0,2.0,0.0,1008.5,1008.5,1008.5,0.0,60.0,60.0,60.0,0.0,2.099609,2.099609,2.099609,0.0,0,4.955327


In [191]:
data['log_meter_reading'] = np.log(data['meter_reading']+1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [192]:
train = data[data['date'].dt.month !="12"]

In [193]:
train.columns.values

array(['building_id', 'meter', 'date', 'meter_reading', 'site_id',
       'primary_use', 'square_feet', 'year_built', 'floor_count',
       'air_temperature', 'cloud_coverage', 'dew_temperature',
       'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction',
       'wind_speed', 'hour', 'day', 'weekend', 'month',
       'air_temperature_mean_lag3', 'air_temperature_max_lag3',
       'air_temperature_min_lag3', 'air_temperature_std_lag3',
       'cloud_coverage_mean_lag3', 'cloud_coverage_max_lag3',
       'cloud_coverage_min_lag3', 'cloud_coverage_std_lag3',
       'dew_temperature_mean_lag3', 'dew_temperature_max_lag3',
       'dew_temperature_min_lag3', 'dew_temperature_std_lag3',
       'precip_depth_1_hr_mean_lag3', 'precip_depth_1_hr_max_lag3',
       'precip_depth_1_hr_min_lag3', 'precip_depth_1_hr_std_lag3',
       'sea_level_pressure_mean_lag3', 'sea_level_pressure_max_lag3',
       'sea_level_pressure_min_lag3', 'sea_level_pressure_std_lag3',
       'wind_direction_mean_l

In [194]:
static_rr = ['square_feet', 'floor_count',
       'air_temperature', 'cloud_coverage', 'dew_temperature',
       'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction',
       'wind_speed' , 'air_temperature_mean_lag3', 'air_temperature_max_lag3',
       'air_temperature_min_lag3', 'air_temperature_std_lag3',
       'cloud_coverage_mean_lag3', 'cloud_coverage_max_lag3',
       'cloud_coverage_min_lag3', 'cloud_coverage_std_lag3',
       'dew_temperature_mean_lag3', 'dew_temperature_max_lag3',
       'dew_temperature_min_lag3', 'dew_temperature_std_lag3',
       'precip_depth_1_hr_mean_lag3', 'precip_depth_1_hr_max_lag3',
       'precip_depth_1_hr_min_lag3', 'precip_depth_1_hr_std_lag3',
       'sea_level_pressure_mean_lag3', 'sea_level_pressure_max_lag3',
       'sea_level_pressure_min_lag3', 'sea_level_pressure_std_lag3',
       'wind_direction_mean_lag3', 'wind_direction_max_lag3',
       'wind_direction_min_lag3', 'wind_direction_std_lag3',
       'wind_speed_mean_lag3', 'wind_speed_max_lag3',
       'wind_speed_min_lag3', 'wind_speed_std_lag3', 'time_idx']

In [195]:
static_categoricalss = ['primary_use', 'month', 'meter']

In [196]:
import pytorch_forecasting

In [197]:
train['meter_reading'] = train['meter_reading'].astype(np.float64)
train['log_meter_reading'] = train['log_meter_reading'].astype(np.float64)

In [198]:
train.head(2)

,building_id,meter,date,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,day,weekend,month,air_temperature_mean_lag3,air_temperature_max_lag3,air_temperature_min_lag3,air_temperature_std_lag3,cloud_coverage_mean_lag3,cloud_coverage_max_lag3,cloud_coverage_min_lag3,cloud_coverage_std_lag3,dew_temperature_mean_lag3,dew_temperature_max_lag3,dew_temperature_min_lag3,dew_temperature_std_lag3,precip_depth_1_hr_mean_lag3,precip_depth_1_hr_max_lag3,precip_depth_1_hr_min_lag3,precip_depth_1_hr_std_lag3,sea_level_pressure_mean_lag3,sea_level_pressure_max_lag3,sea_level_pressure_min_lag3,sea_level_pressure_std_lag3,wind_direction_mean_lag3,wind_direction_max_lag3,wind_direction_min_lag3,wind_direction_std_lag3,wind_speed_mean_lag3,wind_speed_max_lag3,wind_speed_min_lag3,wind_speed_std_lag3,time_idx,log_meter_reading
0,1029,0,2016-01-01,246.000,11,Education,152559,1970.0,3.0,-1.8,9.0,-3.2,0.0,1016.0,280.0,1.5,0,1,4,1,-14.703125,-14.703125,-14.703125,NaN,9.0,9.0,9.0,NaN,-16.40625,-16.40625,-16.40625,NaN,2.0,2.0,2.0,NaN,1008.5,1008.5,1008.5,NaN,60.0,60.0,60.0,NaN,2.099609,2.099609,2.099609,NaN,0,5.509388
1,1029,3,2016-01-01,140.929,11,Education,152559,1970.0,3.0,-1.8,9.0,-3.2,0.0,1016.0,280.0,1.5,0,1,4,1,-14.703125,-14.703125,-14.703125,0.0,9.0,9.0,9.0,0.0,-16.40625,-16.40625,-16.40625,0.0,2.0,2.0,2.0,0.0,1008.5,1008.5,1008.5,0.0,60.0,60.0,60.0,0.0,2.099609,2.099609,2.099609,0.0,0,4.955327


In [199]:
train['air_temperature_std_lag3'].fillna(0.0, inplace = True )
train['cloud_coverage_std_lag3'].fillna(0.0, inplace = True )
train['dew_temperature_std_lag3'].fillna(0.0, inplace = True )
train['precip_depth_1_hr_std_lag3'].fillna(0.0, inplace = True )
train['wind_direction_std_lag3'].fillna(0.0, inplace = True )
train['wind_speed_std_lag3'].fillna(0.0, inplace = True )
train['sea_level_pressure_std_lag3'].fillna(0.0, inplace = True )

In [200]:
data['air_temperature_std_lag3'].fillna(0.0, inplace = True )
data['cloud_coverage_std_lag3'].fillna(0.0, inplace = True )
data['dew_temperature_std_lag3'].fillna(0.0, inplace = True )
data['precip_depth_1_hr_std_lag3'].fillna(0.0, inplace = True )
data['wind_direction_std_lag3'].fillna(0.0, inplace = True )
data['wind_speed_std_lag3'].fillna(0.0, inplace = True )
data['sea_level_pressure_std_lag3'].fillna(0.0, inplace = True )

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [201]:
data['building_id'] = data['building_id'].astype("category")
data['site_id'] = data['site_id'].astype("category")
data['meter'] = data['meter'].astype("category")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [202]:
train['building_id'] = train['building_id'].astype("category")
train['site_id'] = train['site_id'].astype("category")
train['meter'] = train['meter'].astype("category")

In [203]:
train.shape

(119459, 50)

In [204]:
data.shape

(119459, 50)

In [205]:
static_categoricalss

['primary_use', 'month', 'meter']

In [206]:
train['primary_use'] = train['primary_use'].astype("category")

In [207]:
data['primary_use'] = data['primary_use'].astype("category")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [208]:
train['building_id'].dtypes

CategoricalDtype(categories=['1028', '1029', '1030', '1031', '1032'], ordered=False)

In [209]:

max_prediction_length = 1
max_encoder_length = 6
training_cutoff = train["time_idx"].max() - max_prediction_length

In [212]:


training = TimeSeriesDataSet(
    train[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target= "log_meter_reading",
    group_ids=["building_id", "site_id"],
    min_encoder_length=max_encoder_length // 2,  # keep encoder length long (as it is in the validation set)
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length= max_prediction_length,
    static_categoricals= ["building_id", "site_id"],
    static_reals= [],
    time_varying_known_categoricals = ['month'],
    time_varying_known_reals = static_rr,
    time_varying_unknown_categoricals=['primary_use', 'meter'],
    time_varying_unknown_reals=[],
    target_normalizer=GroupNormalizer(
        groups=["building_id", "site_id"], transformation="softplus"
    ),  # use softplus and normalize by group
    categorical_encoders=
    {'meter': pytorch_forecasting.data.encoders.NaNLabelEncoder(add_nan=True),
     'month': pytorch_forecasting.data.encoders.NaNLabelEncoder(add_nan=True),
     'primary_use': pytorch_forecasting.data.encoders.NaNLabelEncoder(add_nan=True),
     'building_id': pytorch_forecasting.data.encoders.NaNLabelEncoder(add_nan=True),
     'site_id': pytorch_forecasting.data.encoders.NaNLabelEncoder(add_nan=True)},
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps = True


)
validation = TimeSeriesDataSet.from_dataset(training, train, predict=True, stop_randomization=True)

/home/ubuntu/.local/lib/python3.6/site-packages/pytorch_forecasting/data/encoders.py:121: UserWarning: Found 5 unknown classes which were set to NaN
  UserWarning,
/home/ubuntu/.local/lib/python3.6/site-packages/pytorch_forecasting/data/encoders.py:121: UserWarning: Found 1 unknown classes which were set to NaN
  UserWarning,
/home/ubuntu/.local/lib/python3.6/site-packages/pytorch_forecasting/data/encoders.py:121: UserWarning: Found 11 unknown classes which were set to NaN
  UserWarning,
/home/ubuntu/.local/lib/python3.6/site-packages/pytorch_forecasting/data/encoders.py:121: UserWarning: Found 3 unknown classes which were set to NaN
  UserWarning,
/home/ubuntu/.local/lib/python3.6/site-packages/pytorch_forecasting/data/encoders.py:121: UserWarning: Found 5 unknown classes which were set to NaN
  UserWarning,
/home/ubuntu/.local/lib/python3.6/site-packages/pytorch_forecasting/data/encoders.py:121: UserWarning: Found 1 unknown classes which were set to NaN
  UserWarning,
/home/ubuntu/.l

In [213]:
batch_size = 128  # set this between 32 to 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)

In [214]:
batch_size

128

In [215]:
actuals = torch.cat([y for x, (y, weight) in iter(val_dataloader)])
baseline_predictions = Baseline().predict(val_dataloader)



In [216]:
from torchmetrics import MeanSquaredError
mse = MeanSquaredError()

In [217]:
mse(actuals, baseline_predictions)

tensor(5.9177)

In [219]:
actuals

tensor([[4.5060],
        [5.4332],
        [3.9826],
        [5.2730],
        [5.3107]])

In [224]:
# configure network and trainer
pl.seed_everything(42)
trainer = pl.Trainer(
    gpus=0,
    # clipping gradients is a hyperparameter and important to prevent divergance
    # of the gradient for recurrent neural networks
    gradient_clip_val=0.1
)


tft = TemporalFusionTransformer.from_dataset(
    training,
    # not meaningful for finding the learning rate but otherwise very important
    learning_rate=0.03,
    hidden_size=1,  # most important hyperparameter apart from learning rate
    # number of attention heads. Set to up to 4 for large datasets
    attention_head_size=4,
    dropout=0.1,  # between 0.1 and 0.3 are good values
    # hidden_continuous_size=4,  # set to <= hidden_size
    output_size=7,  # 7 quantiles by default
    loss=QuantileLoss(),
    # reduce learning rate if no improvement in validation loss after x epochs
#     reduce_on_plateau_patience=4
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

Global seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Number of parameters in network: 3.9k


/home/ubuntu/.local/lib/python3.6/site-packages/torch/nn/init.py:388: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


In [225]:
# # find optimal learning rate
# res = trainer.tuner.lr_find(
#     tft,
#     train_dataloaders=train_dataloader,
#     val_dataloaders=val_dataloader,
#     max_lr=10.0,
#     min_lr=1e-6,
# )

# print(f"suggested learning rate: {res.suggestion()}")
# fig = res.plot(show=True, suggest=True)
# fig.show()

In [249]:

# configure network and trainer
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
lr_logger = LearningRateMonitor()  # log the learning rate
logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard

trainer = pl.Trainer(
    max_epochs=3,
    gpus=0,
    weights_summary="top",
#     gradient_clip_val=0.1,
    limit_train_batches=10,  # coment in for training, running valiation every 30 batches
    # fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
    callbacks=[lr_logger, early_stop_callback],
#     logger=logger,
)


tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.03,
    hidden_size=3,
    attention_head_size=1,
    dropout=0.1,
    hidden_continuous_size=2,
    output_size=7,  # 7 quantiles by default
    loss=QuantileLoss(),
#     log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
#     reduce_on_plateau_patience=4
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Number of parameters in network: 6.3k


In [250]:

# fit network
trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)


   | Name                               | Type                            | Params
----------------------------------------------------------------------------------------
0  | loss                               | QuantileLoss                    | 0     
1  | logging_metrics                    | ModuleList                      | 0     
2  | input_embeddings                   | MultiEmbedding                  | 70    
3  | prescalers                         | ModuleDict                      | 168   
4  | static_variable_selection          | VariableSelectionNetwork        | 275   
5  | encoder_variable_selection         | VariableSelectionNetwork        | 2.8 K 
6  | decoder_variable_selection         | VariableSelectionNetwork        | 2.7 K 
7  | static_context_variable_selection  | GatedResidualNetwork            | 54    
8  | static_context_initial_hidden_lstm | GatedResidualNetwork            | 54    
9  | static_context_initial_cell_lstm   | GatedResidualNetwork            | 54  

Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42
/home/ubuntu/.local/lib/python3.6/site-packages/pytorch_lightning/trainer/data_loading.py:433: UserWarning: The number of training samples (10) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [260]:
import os 
os.getcwd()

'/home/ubuntu'

In [261]:
best_model_path = trainer.checkpoint_callback.best_model_path

In [262]:
best_model_path

'/home/ubuntu/lightning_logs/version_9/checkpoints/epoch=2-step=29.ckpt'

In [263]:
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

In [264]:
best_tft

TemporalFusionTransformer(
  (loss): QuantileLoss()
  (logging_metrics): ModuleList(
    (0): SMAPE()
    (1): MAE()
    (2): RMSE()
    (3): MAPE()
  )
  (input_embeddings): MultiEmbedding(
    (embeddings): ModuleDict(
      (meter): Embedding(4, 3, padding_idx=0)
      (month): Embedding(12, 3, padding_idx=0)
      (primary_use): Embedding(2, 1, padding_idx=0)
      (building_id): Embedding(6, 3, padding_idx=0)
      (site_id): Embedding(2, 1, padding_idx=0)
    )
  )
  (prescalers): ModuleDict(
    (encoder_length): Linear(in_features=1, out_features=2, bias=True)
    (log_meter_reading_center): Linear(in_features=1, out_features=2, bias=True)
    (log_meter_reading_scale): Linear(in_features=1, out_features=2, bias=True)
    (square_feet): Linear(in_features=1, out_features=2, bias=True)
    (floor_count): Linear(in_features=1, out_features=2, bias=True)
    (air_temperature): Linear(in_features=1, out_features=2, bias=True)
    (cloud_coverage): Linear(in_features=1, out_features

In [265]:
# load the best model according to the validation loss
# (given that we use early stopping, this is not necessarily the last epoch)
best_model_path = trainer.checkpoint_callback.best_model_path
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

In [266]:
actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
predictions = best_tft.predict(val_dataloader)

In [267]:
for x, y in (iter(val_dataloader)):
    print(y)

(tensor([[4.5060],
        [5.4332],
        [3.9826],
        [5.2730],
        [5.3107]]), None)


In [268]:
from torchmetrics import MeanSquaredError
mse = MeanSquaredError()
mse(actuals, baseline_predictions)

tensor(5.9177)

In [274]:
# select last 24 months from data (max_encoder_length is 24)
encoder_data = data[lambda x: x.time_idx > x.time_idx.max() - max_encoder_length]

# select last known data point and create decoder data from it by repeating it and incrementing the month
# in a real world dataset, we should not just forward fill the covariates but specify them to account
# for changes in special days and prices (which you absolutely should do but we are too lazy here)
last_data = data[lambda x: x.time_idx == x.time_idx.max()]
decoder_data = pd.concat(
    [last_data.assign(date=lambda x: x.date + pd.offsets.MonthBegin(i)) for i in range(1, max_prediction_length + 1)],
    ignore_index=True,
)

# add time index consistent with "data"
decoder_data["time_idx"] = decoder_data["date"].dt.year * 12 + decoder_data["date"].dt.month
decoder_data["time_idx"] += encoder_data["time_idx"].max() + 1 - decoder_data["time_idx"].min()

# adjust additional time feature(s)
decoder_data["month"] = decoder_data.date.dt.month.astype(str).astype("category")  # categories have be strings

# combine encoder and decoder data
new_prediction_data = pd.concat([encoder_data, decoder_data], ignore_index=True)

In [275]:
new_raw_predictions, new_x = best_tft.predict(new_prediction_data, mode="raw", return_x=True)

/home/ubuntu/.local/lib/python3.6/site-packages/pytorch_forecasting/data/encoders.py:121: UserWarning: Found 5 unknown classes which were set to NaN
  UserWarning,
/home/ubuntu/.local/lib/python3.6/site-packages/pytorch_forecasting/data/encoders.py:121: UserWarning: Found 1 unknown classes which were set to NaN
  UserWarning,
/home/ubuntu/.local/lib/python3.6/site-packages/pytorch_forecasting/data/encoders.py:121: UserWarning: Found 7 unknown classes which were set to NaN
  UserWarning,
/home/ubuntu/.local/lib/python3.6/site-packages/pytorch_forecasting/data/encoders.py:121: UserWarning: Found 3 unknown classes which were set to NaN
  UserWarning,


In [296]:
new_x["decoder_target"].shape

torch.Size([5, 1])

In [299]:
torch.mean(new_raw_predictions["prediction"], dim= 2).shape

torch.Size([5, 1])

In [300]:
mse = MeanSquaredError()
mse(new_x["decoder_target"], torch.mean(new_raw_predictions["prediction"], dim= 2))

tensor(0.7705)